In [15]:
import pandas as pd
from spoiler_generation.utils.dataset_class import Dataset
from datasets import load_dataset

dataset = load_dataset(
    "pramitsahoo/spoiler_generation",
    data_files={"train": "clickbait_spoiling_data/train.jsonl", "validation": "clickbait_spoiling_data/validation.jsonl"},
)

Found cached dataset json (/home/ai23mtech14004/.cache/huggingface/datasets/MateuszW___json/MateuszW--spoiler_generation-ed6e9de16c131595/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 2/2 [00:00<00:00, 236.17it/s]


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags'],
        num_rows: 800
    })
})

In [17]:
train_dataset = Dataset(pd.DataFrame(dataset["train"]), explode=False)

In [18]:
validation_dataset = Dataset(pd.DataFrame(dataset["validation"]), explode=False)

## Add generated questions

In [19]:
questions = load_dataset(
    "MateuszW/spoiler_generation",
    data_files={"train": "generated_questions/train_questions.csv", "validation": "generated_questions/validation_questions.csv"},
)

Found cached dataset csv (/home/ai23mtech14004/.cache/huggingface/datasets/MateuszW___csv/MateuszW--spoiler_generation-714d077ad238af8c/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 2/2 [00:00<00:00, 510.66it/s]


In [20]:
questions

DatasetDict({
    train: Dataset({
        features: ['generated_questions'],
        num_rows: 3200
    })
    validation: Dataset({
        features: ['generated_questions'],
        num_rows: 800
    })
})

In [21]:
train_dataset._df["postText"] = train_dataset._df["postText"] + ". " + questions["train"]["generated_questions"]

In [22]:
validation_dataset._df["postText"] = validation_dataset._df["postText"] + ". " + questions["validation"]["generated_questions"]

## Prepare training/validation data

In [23]:
return_data = train_dataset.prepare_dataset_for_qa_train(save_path="train.json", create_test_dataset=False)

In [24]:
len(return_data)

3143

In [25]:
return_data_val = validation_dataset.prepare_dataset_for_qa_train(save_path="validation.json", create_test_dataset=False)

In [26]:
len(return_data_val)

784

## Prepare test data

In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ai23mtech14004/nltk_data...


True

In [13]:
return_data = train_dataset.prepare_dataset_for_qa_train(save_path="test.json", create_test_dataset=True)

In [14]:
len(return_data)

3200